# 8. IMDB Movie Review Sentiment Analysis

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets
import random
from torchtext.data import TabularDataset

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("cpu와 cuda 중 다음 기기로 학습함:", DEVICE)

SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

cpu와 cuda 중 다음 기기로 학습함: cpu


## 8-1. Review Data 수집 및 전처리

In [2]:
import urllib.request

urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x2d818d6d2c8>)

In [3]:
import pandas as pd

df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [4]:
print('전체 샘플의 개수 : {}'.format(len(df)))

전체 샘플의 개수 : 50000


In [5]:
# PC 사양에 비해 Sample 수가 많으므로, 200개로 줄여 새로운 파일을 만든 후 사용
df = pd.read_csv('IMDb_Reviews_200.csv', encoding='latin1')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [6]:
print('전체 샘플의 개수 : {}'.format(len(df)))

전체 샘플의 개수 : 200


### 훈련 데이터, 평가 데이터, 테스트 데이터로 분리

In [7]:
train_df = df[:150]
test_df = df[150:]

In [8]:
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

## 8-2. Field 정의 및 Dataset 만들기

In [19]:
TEXT = data.Field(sequential=True,lower=True, batch_first=True)
LABEL = data.Field(sequential=False, batch_first=True)

In [10]:
trainset, testset = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

### Dataset sample 확인

In [11]:
print('훈련 샘플의 개수 : {}'.format(len(trainset)))
print('테스트 샘플의 개수 : {}'.format(len(testset)))

훈련 샘플의 개수 : 150
테스트 샘플의 개수 : 50


## 8-3. Vocabulary set 만들기

In [12]:
TEXT.build_vocab(trainset, min_freq=5) # 단어 집합 생성
LABEL.build_vocab(trainset)

In [13]:
n_vocab = len(TEXT.vocab)
print('단어 집합의 크기 : {}'.format(n_vocab))

단어 집합의 크기 : 847


In [14]:
# 훈련 데이터와 평가 데이터 분리
trainset_2, valset = trainset.split(split_ratio=0.75)

In [15]:
print('훈련 샘플의 개수 : {}'.format(len(trainset_2)))
print('평가 샘플의 개수 : {}'.format(len(valset)))

훈련 샘플의 개수 : 112
평가 샘플의 개수 : 38


## 8-4. Data loader

### Dataloader test

In [16]:
BATCH_SIZE = 5

train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset_2, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False, sort=False) 
        # sort=False를 하지 않으면   '<' not supported ... Error 발생할 수 있음.

In [17]:
print('훈련 데이터의 미니 배치의 개수 : {}'.format(len(train_iter)))
print('검증 데이터의 미니 배치의 개수 : {}'.format(len(val_iter)))
print('테스트 데이터의 미니 배치의 개수 : {}'.format(len(test_iter)))

훈련 데이터의 미니 배치의 개수 : 23
검증 데이터의 미니 배치의 개수 : 8
테스트 데이터의 미니 배치의 개수 : 10


In [20]:
print('훈련 데이터의 샘플의 개수 재확인 : {}'.format(len(train_iter.dataset)))
print('검증 데이터의 샘플의 개수 재확인 : {}'.format(len(val_iter.dataset)))
print('테스트 데이터의 샘플의 개수 재확인 : {}'.format(len(test_iter.dataset)))

훈련 데이터의 샘플의 개수 재확인 : 112
검증 데이터의 샘플의 개수 재확인 : 38
테스트 데이터의 샘플의 개수 재확인 : 50


## 8-5. Designing Model

### GRU Modeling

In [21]:
class myModel(nn.Module):
    def __init__(self, n_layers, hidden_size, n_vocab, embed_dim,
                 n_classes, dropout_p):
        super(myModel, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_size,num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_size, n_classes)
                    # n_classes  : 분류되어야 할 결과 수 (긍정 or 부정)

    def forward(self, x):
        x = self.embed(x)                             # (batch 크기 x 문장 단어 크기 x embedding 크기)
        h_0 = self._init_state(batch_size=x.size(0))  # first hidden state 초기화, x.size(0)는 batch 크기
        x_out, _ = self.gru(x, h_0)                   # x_out 벡터 : (batch 크기 x 문장 길이 x 은닉층 크기)
        h_t = x_out[:,-1,:]                           # 마지막 hidden state 선택
        h_t = self.dropout(h_t)
        out = self.out(h_t)                           # (batch 크기 x 분류 갯수)
        return out

    def _init_state(self, batch_size=1):               # hidden state 초기화 정의
        weight = next(self.parameters()).data          # myModel의 가중치 값들을 출력
        return weight.new(self.n_layers, batch_size,  # 가중치 전체 갯수를 유지하면서, 
                          self.hidden_size).zero_()     # hidden state 차원으로 즉, 
                                                       # (층 크기 x batch 크기 x 은닉층 크기)로 바꾸고,
                                                       # 모두 초기화 함.

### hyperparameters

In [22]:
n_layers = 2        # 은닉층 수
hidden_size = 128     # 은닉층 크기
print('단어 집합의 크기 (n_vocab) : {}'.format(n_vocab))
embed_dim= 64      # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
n_classes = 2       # 분류되어야 할 결과 수 (긍정 or 부정)
dropout_p = 0.3
lr = 0.002

단어 집합의 크기 (n_vocab) : 847


In [23]:
model = myModel(n_layers, hidden_size, n_vocab, embed_dim, n_classes, dropout_p).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

## 8-6. Model Training

### Training Code의 간결함을 위해 Model 훈련과 검증을 함수로 정의

In [24]:
def train(model, optimizer, train_iter):
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)            # lable 값 0 또는 1로 조정
        optimizer.zero_grad()
        output = model(x)
        loss = F.cross_entropy(output, y)
        loss.backward()
        optimizer.step()

In [25]:
def evaluate(model, val_iter):
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)
        output = model(x)
        loss = F.cross_entropy(output, y, reduction='sum')
        total_loss += loss.item()                   # loss 누적 값
        corrects += (output.max(1)[1].view(y.size()).data == y.data).sum()   # 정답 맞춘 총횟수
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

### Model Training

In [26]:
EPOCHS = 10
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f" % (e, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

[Epoch: 1] val loss :  0.69 | val accuracy : 55.26
[Epoch: 2] val loss :  0.71 | val accuracy : 44.74
[Epoch: 3] val loss :  0.80 | val accuracy : 42.11
[Epoch: 4] val loss :  0.69 | val accuracy : 57.89
[Epoch: 5] val loss :  0.68 | val accuracy : 47.37
[Epoch: 6] val loss :  0.71 | val accuracy : 52.63
[Epoch: 7] val loss :  0.70 | val accuracy : 44.74
[Epoch: 8] val loss :  0.69 | val accuracy : 52.63
[Epoch: 9] val loss :  0.74 | val accuracy : 44.74
[Epoch: 10] val loss :  0.71 | val accuracy : 60.53


## 8-7. 최적 가중치 확인

In [27]:
model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))    # 저장된 최적 모델 꺼내기
test_loss, test_acc = evaluate(model, test_iter)                        # 평가 진행
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

테스트 오차:  0.72 | 테스트 정확도: 38.00


* Reference  : https://wikidocs.net/60691

    GRU 관련 : https://blog.floydhub.com/gru-with-pytorch/